In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
# Chemin vers tes dossiers : fist, palm, victory
DATA_DIR = "C:/Users/ismai/HandGestureRecognition/data/webcam_gestures"

# 1. Préparation des générateurs avec augmentation de données
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalise les pixels entre 0 et 1
    validation_split=0.2,    # 20% des images pour le test
    rotation_range=20,       # Rotation aléatoire
    zoom_range=0.2,          # Zoom aléatoire
    width_shift_range=0.2,   # Déplacement horizontal
    height_shift_range=0.2,  # Déplacement vertical
    horizontal_flip=False    # Pas de flip car "victory" peut changer de sens
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 840 images belonging to 4 classes.
Found 210 images belonging to 4 classes.


In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 1)),

    # Bloc 1
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Bloc 2
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Bloc 3
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Classification finale
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5), # Désactive 50% des neurones pour éviter l'overfitting
    tf.keras.layers.Dense(3, activation="softmax") # 3 classes : fist, palm, victory
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:

gestures_list = ["fist", "palm", "victory"]

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    classes=gestures_list, # <--- ON FORCE LES 3 CLASSES ICI
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(64, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    classes=gestures_list, # <--- ON FORCE LES 3 CLASSES ICI
    subset="validation"
)

Found 841 images belonging to 3 classes.
Found 210 images belonging to 3 classes.


In [13]:
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen
)


Epoch 1/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.9750 - loss: 0.0896 - val_accuracy: 0.5238 - val_loss: 2.9809
Epoch 2/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.9798 - loss: 0.0585 - val_accuracy: 0.5238 - val_loss: 3.5329
Epoch 3/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.9691 - loss: 0.0775 - val_accuracy: 0.5238 - val_loss: 5.8278
Epoch 4/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.9822 - loss: 0.0547 - val_accuracy: 0.5238 - val_loss: 6.9874
Epoch 5/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.9845 - loss: 0.0495 - val_accuracy: 0.5190 - val_loss: 8.1050
Epoch 6/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.9869 - loss: 0.0487 - val_accuracy: 0.5190 - val_loss: 8.2859
Epoch 7/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - accuracy: 0.9905 - loss: 0.0354 - val_accuracy: 0.5238 - val_loss: 8.9662
Epoch 8/15
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.9798 - loss: 0.0530 - val_accuracy: 0.5238 - v

In [15]:
model.save(
    "C:/Users/ismai/HandGestureRecognition/models/webcam_gesture_cnn.keras"
)
